# MLX

本 Notebook 展示如何开始使用 `MLX` LLM 作为聊天模型。

具体来说，我们将：
1. 使用 [MLXPipeline](https://github.com/langchain-ai/langchain/blob/master/libs/community/langchain_community/llms/mlx_pipeline.py)，
2. 使用 `ChatMLX` 类来使任何这些 LLM 能够与 LangChain 的 [Chat Messages](https://python.langchain.com/docs/modules/model_io/chat/#messages) 抽象进行接口连接。
3. 演示如何使用开源 LLM 来驱动 `ChatAgent` 管道。

In [ ]:
%pip install --upgrade --quiet  mlx-lm transformers huggingface_hub

## 1. 实例化一个 LLM

有三种 LLM 选项可供选择。

In [ ]:
from langchain_community.llms.mlx_pipeline import MLXPipeline

llm = MLXPipeline.from_model_id(
    "mlx-community/quantized-gemma-2b-it",
    pipeline_kwargs={"max_tokens": 10, "temp": 0.1},
)

## 2. 实例化 `ChatMLX` 应用聊天模板

实例化聊天模型和要传递的一些消息。

In [ ]:
from langchain_community.chat_models.mlx import ChatMLX
from langchain_core.messages import HumanMessage

messages = [
    HumanMessage(
        content="What happens when an unstoppable force meets an immovable object?"
    ),
]

chat_model = ChatMLX(llm=llm)

检查聊天消息的 LLM 调用格式。

In [ ]:
chat_model._to_chat_prompt(messages)

调用模型。

In [ ]:
res = chat_model.invoke(messages)
print(res.content)

## 3. 将其作为代理进行试用！

在这里，我们将测试 `gemma-2b-it` 作为零样本 `ReAct` Agent。下面的示例摘自[此处](https://python.langchain.com/docs/modules/agents/agent_types/react#using-chat-models)。

> 注意：要运行此部分，您需要将一个 [SerpAPI Token](https://serpapi.com/) 保存为环境变量：`SERPAPI_API_KEY`

In [ ]:
from langchain import hub
from langchain.agents import AgentExecutor, load_tools
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents.output_parsers import (
    ReActJsonSingleInputOutputParser,
)
from langchain.tools.render import render_text_description
from langchain_community.utilities import SerpAPIWrapper

配置代理，使其使用 `react-json` 风格的提示，并能访问搜索引擎和计算器。

In [ ]:
# setup tools
tools = load_tools(["serpapi", "llm-math"], llm=llm)

# setup ReAct style prompt
# Based on 'hwchase17/react' prompt modification, cause mlx does not support the `System` role
human_prompt = """
Answer the following questions as best you can. You have access to the following tools:

{tools}

The way you use the tools is by specifying a json blob.
Specifically, this json should have a `action` key (with the name of the tool to use) and a `action_input` key (with the input to the tool going here).

The only values that should be in the "action" field are: {tool_names}

The $JSON_BLOB should only contain a SINGLE action, do NOT return a list of multiple actions. Here is an example of a valid $JSON_BLOB:

```
{{
  "action": $TOOL_NAME,
  "action_input": $INPUT
}}
```

ALWAYS use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action:
```
$JSON_BLOB
```
Observation: the result of the action
... (this Thought/Action/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Reminder to always use the exact characters `Final Answer` when responding.

{input}

{agent_scratchpad}

"""

prompt = human_prompt.partial(
    tools=render_text_description(tools),
    tool_names=", ".join([t.name for t in tools]),
)

# define the agent
chat_model_with_stop = chat_model.bind(stop=["\nObservation"])
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_log_to_str(x["intermediate_steps"]),
    }
    | prompt
    | chat_model_with_stop
    | ReActJsonSingleInputOutputParser()
)

# instantiate AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
agent_executor.invoke(
    {
        "input": "Who is Leo DiCaprio's girlfriend? What is her current age raised to the 0.43 power?"
    }
)